In [1]:
import shap
import pickle
from transformers import AutoTokenizer
import numpy as np
import transformers
import shap
from src.plot_text import text
from src.utils import format_fts_for_plotting, text_ft_index_ends
from src.dataset_info import get_dataset_info
from datasets import load_dataset

/home/james/CodingProjects/TextNTabularExplanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Datasets

## Fake 

In [2]:
# with open("../models/shap_vals/fake/shap_vals_ensemble_25.pkl", "rb") as f:
#     fake_25 = pickle.load(f)
with open("../models/shap_vals/fake/shap_vals_ensemble_50.pkl", "rb") as f:
    fake_50 = pickle.load(f)
# with open("../models/shap_vals/fake/shap_vals_ensemble_75.pkl", "rb") as f:
#     fake_75 = pickle.load(f)
with open("../models/shap_vals_ncc/fake/shap_vals_ensemble_50.pkl", "rb") as f:
    ncc_50 = pickle.load(f)


In [3]:
# text_idxs

In [10]:
idx = 1
shap_vals = fake_50
di = get_dataset_info("fake", "weighted")

# To format the text features, we find when the text features end and therefore
# where to insert linebreaks
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
text_idxs = text_ft_index_ends(
    text_fts=shap_vals.data[idx][len(di.tab_cols):], tokenizer=tokenizer
)
linebreak_before_idxs = [len(di.tab_cols)] + [
    x + len(di.tab_cols) + 1 for x in text_idxs
]

formatted_data = np.array(
    format_fts_for_plotting(
        shap_vals[idx].feature_names, shap_vals[idx].data[: len(di.tab_cols)]
    )
)
text(
    shap.Explanation(
        values=shap_vals[idx].values,
        base_values=shap_vals[idx].base_values,
        data=formatted_data,
        clustering=shap_vals[idx].clustering,
        output_names=["Genre is not Drama", "Genre is Drama"],
        hierarchical_values=shap_vals[idx].hierarchical_values,
    ),
    # grouping_threshold=20,
    linebreak_before_idxs=linebreak_before_idxs,  # linebreak_after_idx,
    text_cols=di.text_cols,
    grouping_threshold=1,
)

In [5]:
shap_vals[idx].clustering

array([[0.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e+00],
       [4.90000000e+01, 5.00000000e+01, 2.46913580e-02, 2.00000000e+00],
       [6.40000000e+01, 6.50000000e+01, 2.46913580e-02, 2.00000000e+00],
       [2.00000000e+00, 3.00000000e+00, 2.46913580e-02, 2.00000000e+00],
       [4.00000000e+00, 5.00000000e+00, 2.46913580e-02, 2.00000000e+00],
       [6.00000000e+00, 7.00000000e+00, 2.46913580e-02, 2.00000000e+00],
       [8.00000000e+00, 9.00000000e+00, 2.46913580e-02, 2.00000000e+00],
       [1.00000000e+01, 1.10000000e+01, 2.46913580e-02, 2.00000000e+00],
       [1.20000000e+01, 1.30000000e+01, 2.46913580e-02, 2.00000000e+00],
       [1.40000000e+01, 1.50000000e+01, 2.46913580e-02, 2.00000000e+00],
       [1.80000000e+01, 1.90000000e+01, 2.46913580e-02, 2.00000000e+00],
       [2.00000000e+01, 2.10000000e+01, 2.46913580e-02, 2.00000000e+00],
       [2.20000000e+01, 2.30000000e+01, 2.46913580e-02, 2.00000000e+00],
       [2.40000000e+01, 2.50000000e+01, 2.46913580e

In [6]:
ds = load_dataset(
    di.ds_name, split="test", download_mode="force_redownload"
).to_pandas()
ds = ds.sample(100, random_state=55)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2786.30it/s]
                                                                            

Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [7]:
tokenizer.tokenize()

TypeError: PreTrainedTokenizerFast.tokenize() missing 1 required positional argument: 'text'

In [ ]:
label = 1
num = 250 + 2
print(shap_vals[idx, :, label].values[num])
print(formatted_data[num])


-0.0002450913634583006
opportunities  


In [ ]:
formatted_data[num]

'Flex '

In [ ]:
ds.iloc[:10]

,title,salary_range,description,required_experience,required_education,fraudulent
1371,Senior Strategist,None,Qubit: Cutting Edge Big Data EngineeringQubit ...,5.0,1.0,0
597,Client Relationship Managers,None,Are you looking to start your career in the fi...,-1.0,-1.0,0
1836,Senior Software Tester,80000-120000,VMob is looking for a Senior Tester with exper...,5.0,1.0,0
653,Cook,None,The Cook is responsible for the planning and p...,-1.0,-1.0,0
275,Implementation Manager,80000-100000,We are a growing and profitable software compa...,5.0,10.0,0
2104,Sales Support,30000-40000,Title: Sales Support Type: Permanent – Full-T...,2.0,9.0,0
3087,HR Talent Acquisition Lead,None,Role Summary:HR Talent Acquisition Lead will b...,0.0,1.0,0
657,Customer Service Associate,None,The Customer Service Associate will be based i...,2.0,4.0,0
3117,CSR Specialist,None,Implements the agreed strategy regarding Respo...,5.0,1.0,0
528,Electrical Engineer,None,(We have more than 1500+ Job openings in our w...,-1.0,-1.0,0


In [ ]:
ds.iloc[0]["description"]

'Qubit: Cutting Edge Big Data EngineeringQubit are looking to grow their Client Services Department and we need to find an exceptional analyst / people person to manage a talented group of strategists and junior engineers.You need to have the charm of a concierge, the mental arithmetic skills to win at Countdown and the plate spinning skills of a skilled plate spinner to thrive in this position, owning Qubit’s client relationships, implementing our exceptional technology and coordinating the strategic advice we provide.What you’ll be doingManaging a team of highly intelligent strategists and Customer Solutions Engineers to ensure the client teams objectives are met, from concepts to conversion growth, you’ll be ensuring happiness across the boardConducting analysis and diagnosis of issues and challenges facing our clients ecommerce business. This is where you’ll get to demonstrate your critical thinking skillsIdentifying, planning and formulating strategic opportunities to help clients

In [ ]:
ds


,title,salary_range,description,required_experience,required_education,fraudulent
1371,Senior Strategist,None,Qubit: Cutting Edge Big Data EngineeringQubit ...,5.0,1.0,0
597,Client Relationship Managers,None,Are you looking to start your career in the fi...,-1.0,-1.0,0
1836,Senior Software Tester,80000-120000,VMob is looking for a Senior Tester with exper...,5.0,1.0,0
653,Cook,None,The Cook is responsible for the planning and p...,-1.0,-1.0,0
275,Implementation Manager,80000-100000,We are a growing and profitable software compa...,5.0,10.0,0
...,...,...,...,...,...,...
2804,"Senior Software Engineers, C++",None,Software Competitiveness International (SOFT C...,-1.0,-1.0,0
2767,Social Media Manager,None,Expand and manage Lifehack’s social presence i...,2.0,1.0,0
1203,Chemical Operator,None,DEPARTMENT: Manufacturing/ProductionREPOR...,5.0,-1.0,0
2569,Senior Staff UX Level 3 Designer,None,Why Work for NetConstructor:Join NetConstructo...,-1.0,-1.0,0
